# MIOPY: Use cases

In this tutorial, we demonstrate how MIOPY can be used to study the microRNA/mRNA interaction from expression data.

For this tutorial, we use the TCGA-LUAD dataset.

## Use Case S1: MicroRNAs targeting immune modulators including PD-L1

We were intereseted in finding out which are the most important microRNAs regulating immune-checkpoints in tumor cells.

#### Loading the example dataset

In [1]:
import miopy as mp
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
dfMir, dfRna, metadata = mp.load_dataset()

**We filtered to keep only primary tumor samples**.


In [3]:
dfExpr = mp.concat_matrix(dfMir,dfRna)
dfExpr = dfExpr.loc[metadata.query('sample_type == "PrimaryTumor"').index,:]

#### Run Correlation

In the use case from the publication, we used the Immune Checkpoint (ICBI) geneset, but in this case we reduce the number of genes to reduce the computational times. We can run all the methods with mp.all_methods, every methods can be running indivdually.

In [4]:
lGene = open("genesets/geneset_Immune checkpoints [ICBI].txt","r").read().split()
lGene[0:5]

['PDCD1', 'TNFRSF25', 'TRAF1', 'TRAF2', 'TRAF3']

In [5]:
res, pearson = mp.all_methods(dfExpr, lMirUser = None, lGeneUser = lGene[0:5]+["CD274"], n_core = 8)

Obtain Concat Gene and MIR
Loading dataset...

Classifier Rho

Classifier R

Classifier Tau

Classifier RDC

Classifier Hoeffding

Classifier Ridge

Classifier Lasso

Classifier ElasticNet


As result, the function return a table with all the microRNA/mRNA pairs and the coeficient obatin for each method. 

In [6]:
res.head()

,Mir,Gene,Rho_Pval,Rho,R_Pval,R,Tau_Pval,Tau,RDC,Hoeffding,Ridge,Lasso,ElasticNet,Prediction Tools,Rho_fdr_bh,R_fdr_bh,Tau_fdr_bh
0,hsa-miR-365a-5p,TNFRSF25,0.000066,-0.205621,0.000742,-0.174390,0.000066,-0.138778,0.250640,0.013075,-0.039326,-0.047141,-0.007625,0010000000000000001000010010000000000000,0.001185,0.007370,0.001174
1,hsa-miR-3652,TNFRSF25,0.902039,-0.006412,0.868883,-0.008599,0.909601,-0.003948,0.101052,-0.001282,0.019101,0.019771,0.000000,0110000000000000001000011010000000000100,0.953822,0.931723,0.959374
2,hsa-miR-16-2-3p,TNFRSF25,0.052139,0.100906,0.037187,0.108228,0.056336,0.066366,0.127599,0.001544,-0.075850,-0.036226,0.000000,0000000000000000000000000000000000010000,0.156570,0.121875,0.165623
3,hsa-miR-181c-3p,TNFRSF25,0.147861,0.075279,0.157482,0.073539,0.144025,0.050805,0.158368,0.001204,-0.001540,0.000000,0.000000,0000000000000000000000010000000000000000,0.319107,0.329116,0.311919
4,hsa-miR-210-3p,TNFRSF25,0.000653,0.176182,0.005263,0.144602,0.000618,0.119050,0.197886,0.009523,-0.004650,0.001573,0.002612,0010000000000001000000010010001000000000,0.007062,0.030776,0.006786


**Filtering the results**

Let's now run mp.FilterDF() to keep the most important microRNAs/mRNA pair. FilterDf allow to filter the pairs through the coeficients, the adjust pvalue, and/or the number of prediction tools that predict the interaction. In the publications, we use and FDR < 0.1, coef < -0.3, and min_db > 10.

In [7]:
table, matrix = mp.FilterDF(table = res, matrix = pearson, join = "or", low_coef = -0.2, high_coef = 1, pval = 0.1, analysis = "Correlation", min_db = 10)

MIO implement the BORDA ranking sistem, which use all the metrics in the table to ranking the microRNA/mRNA pairs from the most relevant.

In [8]:
table[["Ranking","Mir","Gene"]].head()

,Ranking,Mir,Gene
0,1.0,hsa-miR-149-5p,TRAF1
1,2.0,hsa-miR-508-5p,TRAF1
2,3.0,hsa-miR-1269a,TRAF3
3,4.0,hsa-miR-1226-3p,TRAF1


#### Predict Target

MIO integrate a custom database from a variety of target prediction tools. In MIO a target prediction can be done using only the 40 integrate prediction tools, or using a gene expression data. In this example, we predict the microRNA whih targeting CD274 (PDL1) using the database, and using the previous results.

**Using only the 40 prediction tools**

In [10]:
table, matrix = mp.predict_target(lTarget = ["CD274",], min_db = 10)

In [17]:
table.sort_values("Number Prediction Tools", ascending=False).head()

,Gene,Mir,Prediction Tools,Number Prediction Tools
4901899,CD274,hsa-miR-138-5p,1110100011000110011001011010011010011100,20
7796711,CD274,hsa-miR-200a-3p,1110100001000110001000011010011110011100,18
8347133,CD274,hsa-miR-20b-5p,1010100001000111001001011010011010011100,18
30323241,CD274,hsa-miR-485-3p,1110100000000010011010011010011010011110,18
580067,CD274,hsa-miR-105-5p,1100100001000010011000011010011010011110,17


**Using the correlation result**

In [21]:
table, matrix = mp.predict_target(table = res, matrix = None, lTarget = ["CD274",], lTools = None, method = "or", min_db = 5, low_coef = -0.2, high_coef = 1, pval = 0.1)
table.sort_values("Ranking").head()

,Mir,Gene,Rho_Pval,Rho,R_Pval,R,Tau_Pval,Tau,RDC,Hoeffding,Ridge,Lasso,ElasticNet,Prediction Tools,Rho_fdr_bh,R_fdr_bh,Tau_fdr_bh,Number Prediction Tools,Ranking
1,hsa-miR-455-3p,CD274,0.0,-0.283,0.0,-0.254,0.0,-0.192,0.279,0.023,-0.096,-0.110,-0.025,0000000000000000001001010000001000010010,0.000,0.0,0.000,6,7.0
2,hsa-miR-149-5p,CD274,0.0,-0.349,0.0,-0.332,0.0,-0.236,0.361,0.035,-0.045,-0.018,-0.026,0010000000000001001000010010001000001000,0.000,0.0,0.000,7,9.0
4,hsa-miR-92a-3p,CD274,0.0,-0.247,0.0,-0.275,0.0,-0.167,0.329,0.018,-0.102,-0.016,-0.034,0010000000000010001000011000001000001000,0.000,0.0,0.000,7,11.0
22,hsa-miR-149-3p,CD274,0.0,-0.221,0.0,-0.230,0.0,-0.148,0.253,0.012,-0.034,-0.047,-0.008,0010000000000000001001010000000000011000,0.000,0.0,0.000,6,31.0
30,hsa-miR-92a-1-5p,CD274,0.0,-0.202,0.0,-0.227,0.0,-0.136,0.312,0.011,-0.037,-0.025,-0.011,0010000000000000001001010010000000010000,0.001,0.0,0.002,6,34.0


## Use Case S2: Genes involved in antigen processing and presentation by microRNAs

Deficient or down regulated genes of the antigen processing and presentation machinery have been associated with response prediction to cancer immunotherapy. In order to study, which microRNAs are potentially able to down regulate the complete pathwey we perfom a correlation analysis using a weigthed expression score.

In [22]:
lGene = open("genesets/geneset_Antigen Processig and Presentation [ImmPort].txt","r").read().split()
dfCor, dfPval, dfSetScore = mp.gene_set_correlation(dfExpr, lGene, GeneSetName = "Antigen Processig and Presentation [ImmPort]", lMirUser = None, n_core = 8)

gene_set_correlation return 3 elements: the pearson's coefficients, the p.value, and the calculate module score for each sample and microRNA.

In [34]:
dfPval.columns = ["P.val"]

In [37]:
table = pd.concat([dfCor, dfPval], axis = 1)

In [40]:
table.sort_values("Antigen Processig and Presentation [ImmPort]").head()

,Antigen Processig and Presentation [ImmPort],P.val
hsa-miR-181a-2-3p,-0.351556,3.115492e-12
hsa-miR-135a-3p,-0.343839,9.815359e-12
hsa-miR-125b-5p,-0.335803,3.137053e-11
hsa-miR-149-5p,-0.326228,1.199474e-10
hsa-miR-130a-3p,-0.316519,4.457801e-10


## Use Case S3: Identifying a microRNA signature predictive for survival

In the publication we used the TCGA-CRC dataset to predict microRNA related with the microsatelite inestability. In this case, we are going to use the TCGA-LUAD to predict the survival (death status) samples. This is only an example about how to use the function.

In [42]:
from miopy.feature_selection import feature_selection

In [48]:
data = pd.concat([dfMir.transpose(),metadata.loc[:,"event"]], axis = 1)
data = data.dropna()

In [60]:
top_feature, dAll, DictScore = feature_selection(data, k = 10, topk = 25, group = "event")

Loading dataset...

Classifier Random Forest
	training: 1.0000, test: 0.6316
	training: 1.0000, test: 0.6316
	training: 1.0000, test: 0.5789
	training: 1.0000, test: 0.5526
	training: 1.0000, test: 0.6579
	training: 1.0000, test: 0.6053
	training: 1.0000, test: 0.5676
	training: 1.0000, test: 0.6486
	training: 1.0000, test: 0.5135
	training: 1.0000, test: 0.5946
	test mean: 0.5982

Classifier Logistic Regresion
	training: 1.0000, test: 0.7105
	training: 1.0000, test: 0.5526
	training: 1.0000, test: 0.5263
	training: 1.0000, test: 0.5000
	training: 1.0000, test: 0.6579
	training: 1.0000, test: 0.5526
	training: 1.0000, test: 0.4865
	training: 1.0000, test: 0.5405
	training: 1.0000, test: 0.5405
	training: 1.0000, test: 0.7297
	test mean: 0.5797

Classifier Ridge Classfier
	training: 1.0000, test: 0.7105
	training: 1.0000, test: 0.6053
	training: 1.0000, test: 0.4211
	training: 1.0000, test: 0.6316
	training: 1.0000, test: 0.5526
	training: 1.0000, test: 0.5789
	training: 1.0000, test: 0

Th feature selection return the top predictors most informative in separating the death status in the TCGA-LUAD patients. Now, we can use this predictors to training a model, and see how robust are these microRNAs.

In [61]:
from miopy.classification import classification_cv

In [62]:
results = classification_cv(data, k = 5, name = "Random Forest", group = "event", lFeature = top_feature.index)


Loading dataset...

Classifier Random Forest
	training: 1.0000, test: 0.7763
	training: 1.0000, test: 0.6933
	training: 1.0000, test: 0.6400
	training: 1.0000, test: 0.6800
	training: 1.0000, test: 0.5600
	Test Mean: 0.6699


## Use Case S4: MicroRNA target genes synthetic lethal to immune (therapy) essential genes

In order to identify synthetic lethal partner genes in tumor cells we have taken advantage of previous efforts and used the ISLE algorithm for calculation (Lee et al., 2018), which is available within MIO. We were
interested in identifying microRNAs targeting genes which are synthetic lethal to immune(therapy) essential genes. We used the option Target Prediction, miRNA Synthetic Lethal Prediction. 

In addition, MIOPY can perform an overrepresentation analysis for microRNAs based on the number of synthetic lethal target genes compared to all potential target genes.

In [63]:
lGene = open("genesets/geneset_Immune essential genes [Patel].txt","r").read().split()

In [66]:
target, matrix, ora = mp.predict_lethality2(lQuery = lGene, lTools = None, method = "or", min_db = 25)

In [68]:
target.sort_values("Number Prediction Tools", ascending=False).head()

,Query,Synthetic Lethal,Gene,Mir,Prediction Tools,Number Prediction Tools
280,MED23,HOXC8,HOXC8,hsa-miR-196a-5p,1111111111101010111110111110101110001111,31
186,RECQL4,BACH1,BACH1,hsa-miR-155-5p,1011101011001110111110111110111110011111,30
157,PPARD,ZFX,ZFX,hsa-miR-144-3p,1111110011101111111111011110001110011110,30
192,RECQL4,BNIP2,BNIP2,hsa-miR-20b-5p,0110111011101111111101111110001110011111,30
281,MED23,HOXC8,HOXC8,hsa-miR-196b-5p,1111111011101011111110111110000110001111,29


In [71]:
ora.sort_values("FDR").head()

,microRNA,Target Number,Expected Number,Fold Enrichment,Raw P-value,FDR
hsa-miR-124-3p,hsa-miR-124-3p,11,2.92598,3.78978,0.000415719,0.0357519
hsa-miR-20b-5p,hsa-miR-20b-5p,18,9.76979,1.85768,0.0215791,0.447906
hsa-miR-373-3p,hsa-miR-373-3p,6,2.47964,2.42818,0.0479099,0.447906
hsa-miR-221-3p,hsa-miR-221-3p,5,1.93412,2.59303,0.0553376,0.447906
hsa-miR-106b-5p,hsa-miR-106b-5p,7,3.91784,1.79219,0.127321,0.447906
